In [2]:
import math
import pickle
import statistics
import numpy as np
import pandas as pd
from collections import defaultdict

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import itertools
from sklearn.decomposition import PCA
from mlxtend.plotting import plot_decision_regions
from mlxtend.classifier import StackingClassifier 

from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler, label_binarize, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, log_loss, make_scorer
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, confusion_matrix, auc
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from xgboost import XGBClassifier
from catboost import CatBoostRegressor

In [277]:
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [3]:
df_drop=pd.read_pickle("Data/df_drop.pkl")
test = pd.read_csv("Data/test_set_features.csv")

In [67]:
df=pd.read_pickle("Data/df.pkl")

In [31]:
df_test=test.iloc[:,1:]

In [47]:
test_id=test.iloc[:,0]

In [281]:
df_train=df.values

X = df_train[:, 3:].astype(str)
y = df_train[:, 1].astype(str) #1=H1N1 2=Seasonal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=4444)


***

In [202]:
# one-hot encode input variables
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder.fit(X)
X = onehot_encoder.transform(X)
# X_test = onehot_encoder.transform(X_test)


In [93]:
from fancyimpute import IterativeImputer 
mice_imputer = IterativeImputer() 
X = mice_imputer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=4444)

***

In [282]:
# one-hot encode input variables
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder.fit(X_train)
X_train = onehot_encoder.transform(X_train)
X_test = onehot_encoder.transform(X_test)


In [283]:
#ordinal encode target variable
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)

In [98]:
# from sklearn.model_selection import RandomizedSearchCV
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

In [102]:
# from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor(random_state = 42)

# # Use the random grid to search for best hyperparameters
# # First create the base model to tune
# rf = RandomForestRegressor()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# # Fit the random search model
# rf_random.fit(X_train, y_train)

In [234]:

model = RandomForestClassifier(n_estimators=17, n_jobs=2)
# fit on the training set
model.fit(X_train, y_train)
# predict on test set
yhat = model.predict(X_test)


In [235]:
# predict on test set
# evaluate predictions
print("Training: {:6.2f}%".format(100*model.score(X_train, y_train)))
print("Test set: {:6.2f}%".format(100*model.score(X_test, y_test)))
accuracy = accuracy_score(y_test, yhat)
roc_auc = roc_auc_score(y_test, yhat)
print('Accuracy_: %.2f' % (accuracy*100))
print('roc_auc_score: %.2f' % (roc_auc*100))
print(classification_report(y_test, yhat))

Training:  99.61%
Test set:  86.07%
Accuracy_: 86.07
roc_auc_score: 74.32
              precision    recall  f1-score   support

           0       0.88      0.95      0.91      4203
           1       0.74      0.54      0.62      1139

    accuracy                           0.86      5342
   macro avg       0.81      0.74      0.77      5342
weighted avg       0.85      0.86      0.85      5342



In [238]:
test_test=df_test.values.astype(str)
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder.fit(test_test)
test_test = onehot_encoder.transform(test_test)

In [275]:
y_s1 = model.predict_proba(test_test)

In [194]:
y_s2 = model.predict_proba(test_test)

In [198]:
y_s1

array([0.18, 0.04, 0.18, ..., 0.16, 0.04, 0.5 ])

In [197]:
y_s2

array([0.42857143, 0.14285714, 0.85714286, ..., 0.42857143, 0.28571429,
       0.85714286])

In [199]:
sub = pd.DataFrame(
    {
    "respondent_id": test_id,
    "h1n1_vaccine": y_s1[:,1],
    "seasonal_vaccine": y_s2[:,1],
    } 
)

In [200]:
sub.to_csv("Data/sub_rf.csv", index=False)

In [262]:
df_test.to_csv("Data/df_test.csv", index=False)

In [248]:
df_sea = df.iloc[:,2:]

In [256]:
df_h1n1 = pd.DataFrame(df.iloc[:,1:]).drop(['seasonal_vaccine'], axis=1)

In [258]:
df_h1n1.to_csv("Data/df_h1n1.csv", index=False)

In [259]:
df_sea.to_csv("Data/df_sea.csv", index=False)

In [265]:
res_h1n1 = pd.read_csv("h1n1_res.csv")

In [268]:
res_h1n1 = res_h1n1["Prediction"]

In [270]:
res_sea = pd.read_csv("sea_res.csv")

In [271]:
res_sea = res_sea["Prediction"]

In [272]:
sub_2 = pd.DataFrame(
    {
    "respondent_id": test_id,
    "h1n1_vaccine": res_h1n1,
    "seasonal_vaccine": res_sea,
    } 
)

In [274]:
sub_2.to_csv("Data/sub_2.csv", index=False)

In [279]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval

In [284]:
def hyperopt_lgb_score(params):
    clf = LGBMClassifier(**params)
    current_score = cross_val_score(clf, X_train, y_train, cv=10).mean()
    print(current_score, params)
    return current_score 
 
space_lgb = {
            'learning_rate': hp.quniform('learning_rate', 0, 0.05, 0.0001),
            'n_estimators': hp.choice('n_estimators', range(100, 1000)),
            'max_depth':  hp.choice('max_depth', np.arange(2, 12, dtype=int)),
            'num_leaves': hp.choice('num_leaves', 2*np.arange(2, 2**11, dtype=int)),
            'min_child_weight': hp.quniform('min_child_weight', 1, 9, 0.025),
            'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.005),
            'objective': 'binary',
            'boosting_type': 'gbdt',
            }
 
best = fmin(fn=hyperopt_lgb_score, space=space_lgb, algo=tpe.suggest, max_evals=10)
print('best:')
print(best)

0.854201762595539                                     
{'boosting_type': 'gbdt', 'colsample_bytree': 0.9400000000000001, 'learning_rate': 0.0397, 'max_depth': 5, 'min_child_weight': 2.6750000000000003, 'n_estimators': 613, 'num_leaves': 1426, 'objective': 'binary'}
0.8503640162010869                                                             
{'boosting_type': 'gbdt', 'colsample_bytree': 0.68, 'learning_rate': 0.0436, 'max_depth': 8, 'min_child_weight': 3.4250000000000003, 'n_estimators': 947, 'num_leaves': 2764, 'objective': 'binary'}
0.8113272000897334                                                               
{'boosting_type': 'gbdt', 'colsample_bytree': 0.725, 'learning_rate': 0.0015, 'max_depth': 10, 'min_child_weight': 5.375, 'n_estimators': 419, 'num_leaves': 182, 'objective': 'binary'}
0.7877369961039575                                                               
{'boosting_type': 'gbdt', 'colsample_bytree': 0.53, 'learning_rate': 0.0009000000000000001, 'max_depth': 4, 

In [285]:
params = space_eval(space_lgb, best)
params

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.53,
 'learning_rate': 0.0009000000000000001,
 'max_depth': 4,
 'min_child_weight': 4.45,
 'n_estimators': 603,
 'num_leaves': 3782,
 'objective': 'binary'}

In [294]:
LGB_Classifier = LGBMClassifier(**params)
LGB_Classifier.fit(X_train, y_train)
Y_pred = LGB_Classifier.predict_proba(test_test).astype(int)
LGB_Classifier.score(X_train, y_train)
acc_LGB_Classifier = round(LGB_Classifier.score(X_train, y_train) * 100, 2)
acc_LGB_Classifier

78.77

In [295]:
Y_pred

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])